# Different visualisation for SupraLaplacian 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np 
from scipy.sparse.linalg import eigsh
from scipy.linalg import block_diag

## Supra-Laplacian with isolated nodes

In [ ]:
G1 =  nx.erdos_renyi_graph(10, 0.4)
G2 =  nx.erdos_renyi_graph(10, 0.4)
G3 = nx.erdos_renyi_graph(10, 0.4)

In [ ]:
# check if G1,G2,G3 are connected
print(nx.is_connected(G1))
print(nx.is_connected(G2))
print(nx.is_connected(G3))

In [ ]:
# Add isolated nodes in G1,G2,G3
G1.add_node(10)
G1.add_node(11)
G1.add_node(12)
G2.add_node(10)
G2.add_node(11)
G2.add_node(12)
G3.add_node(10)
G3.add_node(11)
G3.add_node(12)
# check if G1,G2,G3 are connected
print(nx.is_connected(G1))
print(nx.is_connected(G2))
print(nx.is_connected(G3))
adj_matrix1 = nx.adjacency_matrix(G1).todense()
adj_matrix2 = nx.adjacency_matrix(G2).todense()
adj_matrix3 = nx.adjacency_matrix(G3).todense()
sup_adj_matrix = block_diag(adj_matrix1,adj_matrix2,adj_matrix3)

# generate graph from adj_matrix
sup_G = nx.from_numpy_array(sup_adj_matrix)

In [ ]:
sup_lap = nx.laplacian_matrix(sup_G).toarray().astype(np.float32)
eig_vals, eig_vecs = eigsh(sup_lap,k=69,
                which='SA',
                return_eigenvectors=True,)

## Visu supra laplacien

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
def plot_supragraph_laplacian(sup_G,val_propre,vec_propre,pos):
    cmap = LinearSegmentedColormap.from_list(
        "custom_cmap", ["purple", "white", "green"], N=256)
    fig, ax = plt.subplots(figsize=(8, 8))  # Create a figure and an axes
    nx.draw(sup_G, pos=pos, node_color=vec_propre, cmap=cmap, vmin=-1, vmax=1,edgecolors='black', with_labels=True, ax=ax,node_size=800)
    
    # Create a ScalarMappable object
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin = -1, vmax=1))
    sm.set_array([])
    
    # Add the colorbar to the figure
    fig.colorbar(sm, ax=ax, orientation='vertical')  # Use the axes for the colorbar

## Generate position for the node 

In [ ]:
# Save pickle fixed_layout 
import pickle
import os
if os.path.exists('fixed_layout.pkl'):
    with open('fixed_layout.pkl', 'rb') as f:
        fixed_layout = pickle.load(f)
else:
    fixed_layout = nx.spring_layout(G1,seed=42,k=0.9)
    with open('fixed_layout.pkl', 'wb') as f:
        pickle.dump(fixed_layout, f)


In [ ]:

new_fixed_layout = dict()
N = len(fixed_layout)
for i in range(N):
    y,x = list(fixed_layout.items())[i][1]
    shift = 2.3
    pos2 =  np.array([x+shift,y])
    pos3 = np.array([x+ 2*shift,y ])
    new_fixed_layout[i] = np.array([x,y])
    new_fixed_layout[i+N] = pos2
    new_fixed_layout[i+2*N] = pos3 


## Time connection | Isolated Nodes | No virtual nodes (visu 1)

In [ ]:

for i in range(N):
    sup_G.add_edge(i,i+N)
    sup_G.add_edge(i+N,i+2*N)

In [ ]:
sup_lap = nx.laplacian_matrix(sup_G).toarray().astype(np.float32)
eig_vals, eig_vecs = eigsh(sup_lap,k=69,
                which='SA',
                return_eigenvectors=True,)

In [ ]:
for i in range(39):
    plot_supragraph_laplacian(sup_G,eig_vals,eig_vecs[:,i],new_fixed_layout)
    plt.title(f"{i} th eigen vector with eigen value {eig_vals[i]:.2f}") 
    plt.savefig(f"Visu/Visu1/eigen_{i}.png")
    plt.show()
    plt.close()

## Virtual nodes + time connections (visu2)

In [ ]:
len(sup_G.nodes)

In [ ]:
N = len(sup_G.nodes)
for i in range(3):
    sup_G.add_node(N+i)

In [ ]:
for i in range(int(N/3)):
    sup_G.add_edge(i,N)
    sup_G.add_edge(i+int(N/3),N+1)
    sup_G.add_edge(i+2*int(N/3),N+2)

In [ ]:
sup_lap = nx.laplacian_matrix(sup_G).toarray().astype(np.float32)
eig_vals, eig_vecs = eigsh(sup_lap,k=69,
                which='SA',
                return_eigenvectors=True,)

In [ ]:
len(eig_vals)

In [ ]:
new_fixed_layout[N] = np.array([0.05,0.5])
new_fixed_layout[N+1] = np.array([2.3,0.5])
new_fixed_layout[N+2] = np.array([4.6,0.5])

In [ ]:

for i in range(len(sup_G.nodes)):
    plot_supragraph_laplacian(sup_G,eig_vals,eig_vecs[:,i],new_fixed_layout)
    plt.title(f"{i} th eigen vector with eigen value {eig_vals[i]:.2f}") 
    plt.savefig(f"Visu/Visu2/eigen_{i}.png")
    plt.show()
    plt.close()

## No isolated nodes , virtual nodes and temporal connections (SLATE method) (visu 3). 

In [ ]:
sup_G.remove_node(10)
sup_G.remove_node(11)
sup_G.remove_node(12)
sup_G.remove_node(23)
sup_G.remove_node(24)
sup_G.remove_node(25)
sup_G.remove_node(36)
sup_G.remove_node(37)
sup_G.remove_node(38)

In [ ]:
sup_lap = nx.laplacian_matrix(sup_G).toarray().astype(np.float32)
eig_vals, eig_vecs = eigsh(sup_lap,k=69,
                which='SA',
                return_eigenvectors=True,)

In [ ]:
for i in range(len(sup_G.nodes)):
    plot_supragraph_laplacian(sup_G,eig_vals,eig_vecs[:,i],new_fixed_layout)
    plt.title(f"{i} th eigen vector with eigen value {eig_vals[i]:.2f}") 
    plt.savefig(f"Visu/Visu3/eigen_{i}.png")
    plt.show()
    plt.close()

## Connected graphs without isolated nodes and with temporal connections (Visu4)

In [ ]:
sup_G.remove_node(39)
sup_G.remove_node(40)
sup_G.remove_node(41)

In [ ]:
sup_lap = nx.laplacian_matrix(sup_G).toarray().astype(np.float32)
eig_vals, eig_vecs = eigsh(sup_lap,k=69,
                which='SA',
                return_eigenvectors=True,)

In [ ]:
for i in range(len(sup_G.nodes)):
    plot_supragraph_laplacian(sup_G,eig_vals,eig_vecs[:,i],new_fixed_layout)
    plt.title(f"{i} th eigen vector with eigen value {eig_vals[i]:.2f}") 
    plt.savefig(f"Visu/Visu4/eigen_{i}.png")
    plt.show()
    plt.close()

## No temporal connection no isolated nodes (Visu5)

In [ ]:
# remove temporal connections
for i in range(10):
    sup_G.remove_edge(i,i+13)
    sup_G.remove_edge(i+13,i+26)

In [ ]:
sup_lap = nx.laplacian_matrix(sup_G).toarray().astype(np.float32)
eig_vals, eig_vecs = eigsh(sup_lap,k=69,
                which='SA',
                return_eigenvectors=True,)

In [ ]:
for i in range(len(sup_G.nodes)):
    plot_supragraph_laplacian(sup_G,eig_vals,eig_vecs[:,i],new_fixed_layout)
    plt.title(f"{i} th eigen vector with eigen value {eig_vals[i]:.2f}") 
    plt.savefig(f"Visu/Visu5/eigen_{i}.png")
    plt.show()
    plt.close()